In [58]:
import os
import json
import pandas as pd
import traceback

In [59]:
from langchain_openai import ChatOpenAI

In [60]:
from dotenv import load_dotenv
load_dotenv()

True

In [61]:
KEY=os.getenv('OPENAI_API_KEY')

In [62]:
llm=ChatOpenAI(openai_api_key=KEY, model_name="gpt-3.5-turbo", temperature=0.3)

In [63]:
from langchain_openai import OpenAI  # Use this for OpenAI models
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain_community.callbacks.manager import get_openai_callback  # New callback import
import PyPDF2


In [64]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [65]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [66]:
quiz_generation_prompt = PromptTemplate(
    input_variables = ['text','number','subject', 'tone', 'response_json'],
    template=TEMPLATE
)

In [67]:
quiz_chain = LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key='quiz', verbose=True)

In [68]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [69]:
quiz_evaluation_prompt = PromptTemplate(input_variables = ['subject', 'quiz'], template=TEMPLATE2)

In [70]:
review_chain = LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key='review', verbose=True)

In [71]:
generate_evaluate_chain = SequentialChain(chains=[quiz_chain, review_chain], input_variables=['text','number','subject', 'tone', 'response_json'],output_variables=['quiz','review'], verbose=True)

In [72]:
file_path = r"C:\Users\Asus\mcqgen\data.txt"

In [73]:
with open(file_path, 'r') as file:
    TEXT = file.read()

In [74]:
print(TEXT)

The company was founded in 2016 by French entrepreneurs Clément Delangue, Julien Chaumond, and Thomas Wolf in New York City, originally as a company that developed a chatbot app targeted at teenagers.[2] The company was named after the U+1F917 🤗 HUGGING FACE emoji.[2] After open sourcing the model behind the chatbot, the company pivoted to focus on being a platform for machine learning.

In March 2021, Hugging Face raised US$40 million in a Series B funding round.[3]

On April 28, 2021, the company launched the BigScience Research Workshop in collaboration with several other research groups to release an open large language model.[4] In 2022, the workshop concluded with the announcement of BLOOM, a multilingual large language model with 176 billion parameters.[5][6]

In December 2022, the company acquired Gradio, an open source library built for developing machine learning applications in Python.[7]

On May 5, 2022, the company announced its Series C funding round led by Coatue and Seq

In [75]:
# Serialize the python dictionary into JSON-formatted string
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [76]:
NUMBER = 3
SUBJECT = "Hugging Face"
TONE = "simple"

In [77]:
#https://python.langchain.com/docs/modules/model_io/llms/token_usage_tracking

#How to setup Token Usage Tracking in LangChain
with get_openai_callback() as cb:
    response=generate_evaluate_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject":SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
        )



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:The company was founded in 2016 by French entrepreneurs Clément Delangue, Julien Chaumond, and Thomas Wolf in New York City, originally as a company that developed a chatbot app targeted at teenagers.[2] The company was named after the U+1F917 🤗 HUGGING FACE emoji.[2] After open sourcing the model behind the chatbot, the company pivoted to focus on being a platform for machine learning.

In March 2021, Hugging Face raised US$40 million in a Series B funding round.[3]

On April 28, 2021, the company launched the BigScience Research Workshop in collaboration with several other research groups to release an open large language model.[4] In 2022, the workshop concluded with the announcement of BLOOM, a multilingual large language model with 176 billion parameters.[5][6]

In December 2022, the company acquired Gradio, an open source library built for developing machine learning applic

In [78]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

Total Tokens:1365
Prompt Tokens:1130
Completion Tokens:235
Total Cost:0.0009175


In [79]:
response

{'text': "The company was founded in 2016 by French entrepreneurs Clément Delangue, Julien Chaumond, and Thomas Wolf in New York City, originally as a company that developed a chatbot app targeted at teenagers.[2] The company was named after the U+1F917 🤗 HUGGING FACE emoji.[2] After open sourcing the model behind the chatbot, the company pivoted to focus on being a platform for machine learning.\n\nIn March 2021, Hugging Face raised US$40 million in a Series B funding round.[3]\n\nOn April 28, 2021, the company launched the BigScience Research Workshop in collaboration with several other research groups to release an open large language model.[4] In 2022, the workshop concluded with the announcement of BLOOM, a multilingual large language model with 176 billion parameters.[5][6]\n\nIn December 2022, the company acquired Gradio, an open source library built for developing machine learning applications in Python.[7]\n\nOn May 5, 2022, the company announced its Series C funding round led

In [80]:
quiz=response.get('quiz')

In [81]:
quiz=json.loads(quiz)

In [82]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [83]:
quiz_table_data

[{'MCQ': 'When was Hugging Face founded?',
  'Choices': 'a: 2014 | b: 2016 | c: 2018 | d: 2020',
  'Correct': 'b'},
 {'MCQ': 'What was the original focus of Hugging Face before pivoting to machine learning?',
  'Choices': 'a: Social media platform | b: E-commerce website | c: Chatbot app targeted at teenagers | d: Gaming platform',
  'Correct': 'c'},
 {'MCQ': 'Which company led the Series D funding round for Hugging Face in August 2023?',
  'Choices': 'a: Google | b: Salesforce | c: Amazon | d: Nvidia',
  'Correct': 'b'}]

In [84]:
quiz=pd.DataFrame(quiz_table_data)

In [85]:
quiz.to_csv("HuggingFace.csv", index=False)

In [86]:
from datetime import datetime
datetime.now()

datetime.datetime(2024, 10, 4, 8, 29, 47, 540702)